In [1]:
using Plots, TaylorSeries
pyplot(legend = :false)

Plots.PyPlotBackend()

Ok, primero hay que lidiar con las condiciones iniciales, establecemos las constantes necesarias:

In [2]:
const λ = 1.0
const Z = 2.0
const _ordenTaylor = 28
const _epsAbs = 1.0e-20


1.0e-20

Para establecer condiciones iniciales, debemos tomar aquellas que cumplan con que $E = -1$, además de que los electrones deben estar en el eje $x$, además: $p_{\theta} = 0$

In [3]:
# Returns stepsize of the integration and a vector with the updated values of the dependent
# variables
function taylorStepper{T<:Real}( jetEqs::Function, vec0::Array{T,1} )
    
    n = length( vec0 )

    vec0T = Array(Taylor1{T},n)
    @simd for i in eachindex(vec0)
        @inbounds vec0T[i] = Taylor1([vec0[i]], _ordenTaylor)
    end

    # Jets
    vec1T = jetEqs( vec0 )
    
    # Step-size
    hh = Inf
    for i in eachindex(vec1T)
        @inbounds h1 = stepsize( vec1T[i], _epsAbs )
        hh = min( hh, h1 )
    end
    
    # Values at t0+h
    @simd for i in eachindex(vec0)
        @inbounds vec0[i] = evaluate( vec1T[i], hh )
    end
    
    return hh, vec0
end

# Returns the maximum step size from epsilon and the last two coefficients of the x-Taylor series 
function stepsize{T<:Real}(x::Taylor1{T}, epsilon::Float64)
    ord = x.order
    h = Inf
    for k in [ord-1, ord]
        kinv = 1.0/k
        aux = abs( x.coeffs[k+1] )
        h = min(h, (epsilon/aux)^kinv)
    end
    return h
end

stepsize (generic function with 1 method)

Aquí implementaremos el hamiltoniano regularizado, con $E = -1$, debe cumplirse que $H = 0 \ \forall t$, primero, tomaremos una función auxiliar:

In [9]:
function ks(Q)
    Q₁, Q₂ = Q
    x₁ = Q₁^2 - Q₂^2
    x₂ = 2Q₁*Q₂
    return Float64[Q₁, Q₂]
end

ks (generic function with 1 method)

In [16]:
function regHam(Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = norm(P₁)^2
    P₂² = norm(P₂)^2
    Q₁² = norm(Q₁)^2
    Q₂² = norm(Q₂)^2
    nf = norm(ks(Q₁) - ks(Q₂))
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end

regHam(Q11, Q12, P11, P12, Q21, Q22, P21, P22) = regHam([Q11, Q12],[Q21, Q22],[P11, P12],[P21, P22])

regHam (generic function with 2 methods)

Implementaremos también la función para regresar las velocidades al espacio fase de la configuración original... 

In [10]:
function ksv(Q, P)
    Q₁, Q₂ = Q
    P₁, P₂ = P
    v₁ = (Q₁*P₁ - Q₂*P₂)/(2*norm(Q)^2)
    v₂ = (Q₂*P₁ + Q₁*P₂)/(2*norm(Q)^2)
    return Float64[v₁, v₂]
end

ksv (generic function with 1 method)

In [ ]:
function keplerIntegration(Q₁, Q₂, P₁, P₂, time_max::Float64, jetEqs::Function )
    τ₀ = 0.0
    t0 = 0.0
    Q11₀, Q12₀ = Q₁
    P11₀, P12₀ = P₁
    Q21₀, Q22₀ = Q₂
    P21₀, P22₀ = P₂
    
    ene0 = regHam(Q₁, Q₂, P₁, P₂)
        
    Q11v, Q12v, P11v, P12v = Float64[], Float64[], Float64[], Float64[]
    Q21, Q22, P21v, P22v = Float64[], Float64[], Float64[], Float64[]
    tv, τv = Float64[], Float64[]
    DeneV= Float64[]
    push!(tv, t0)
    push!(Q11v, Q11₀)
    push!(Q12v, Q12₀)
    push!(P11v, P11₀)
    push!(P12v, P12₀)
    push!(Q21v, Q21₀)
    push!(Q22v, Q22₀)
    push!(P21v, P21₀)
    push!(P22v, P22₀)
       
    push!(DeneV, zero(Float64))
    
    # This is the main loop; we include a minimum step size for security
    dτ = 1.0
    while dτ < time_max && dτ>1.0e-8
        #Preparamos el vector para integrar
        temp1 = Float64[t0, Q11₀, Q12₀, P11₀, P12₀, Q21₀, Q22₀, P21₀, P22₀]
        # Here we integrate
        dτ, (t, Q11, Q12, P11, P12, Q21, Q22, P21, P22) = taylorStepper( jetEqs, temp1);
        
        Q11, Q12 = Q₁
        P11, P12 = P₁
        Q21, Q22 = Q₂
        P21, P22 = P₂
        τ0 += dτ
        push!(tv, t)
        push!(τv, τ0)
        push!(Q11v, Q11)
        push!(Q12v, Q12)
        push!(P11v, P11)
        push!(P12v, P12)
        push!(Q21v, Q21)
        push!(Q22v, Q22)
        push!(P21v, P21)
        push!(P22v, P22)
        eneEnd = regHam(Q11, Q12, P11, P12, Q21, Q22, P21, P22)
        
        dEne = abs(eneEnd)
        
        push!(DeneV, dEne)
        temp1 = Float64[t, Q11, Q12, P11, P12, Q21, Q22, P21, P22]
    end

    return tv, τv, Q11v, Q12v, P11v, P12v, Q21v, Q22v, P21v, P22v, DeneV
end

In [ ]:
function jetundriven{T<:Real}( vec::Array{T,1} )

    tT = Taylor1(vec[1], _ordenTaylor)
    Q11T = Taylor1(vec[2], _ordenTaylor)
    Q12T = Taylor1(vec[3], _ordenTaylor)
    P11T = Taylor1(vec[4], _ordenTaylor)
    P12T = Taylor1(vec[5], _ordenTaylor)
    Q21T = Taylor1(vec[6], _ordenTaylor)
    Q22T = Taylor1(vec[7], _ordenTaylor)
    P21T = Taylor1(vec[8], _ordenTaylor)
    P22T = Taylor1(vec[9], _ordenTaylor)
    # Auxiliary quantities
    x2T = zeros( T, _ordenTaylor+1 )
    y2T = zeros( T, _ordenTaylor+1 )
    sT  = zeros( T, _ordenTaylor+1 )
    rT3 = zeros( T, _ordenTaylor+1 )
    nQ1 = zeros( T, _ordenTaylor+1 )
    nQ2 = zeros( T, _ordenTaylor+1 )
    nP1 = zeros( T, _ordenTaylor+1 )
    nP2 = zeros( T, _ordenTaylor+1 )
    n2Q1 = zeros( T, _ordenTaylor+1 )
    n2Q2 = zeros( T, _ordenTaylor+1 )
    n2P1 = zeros( T, _ordenTaylor+1 )
    n2P2 = zeros( T, _ordenTaylor+1 )
    dotQ11 = zeros( T, _ordenTaylor+1 )
    dotQ12 = zeros( T, _ordenTaylor+1 )
    dotP11 = zeros( T, _ordenTaylor+1 )
    dotP12 = zeros( T, _ordenTaylor+1 )
    dotQ21 = zeros( T, _ordenTaylor+1 )
    dotQ22 = zeros( T, _ordenTaylor+1 )
    dotP21 = zeros( T, _ordenTaylor+1 )
    dotP22 = zeros( T, _ordenTaylor+1 )

    # Now the implementation
    for k = 0:_ordenTaylor-1
        knext = k+1
        # The right-hand size of the eqs of motion
        # This is more adpated for this problem, and avoids many superflous operations
        x2T[knext] = TaylorSeries.squareHomogCoef(k, xT.coeffs)
        y2T[knext] = TaylorSeries.squareHomogCoef(k, yT.coeffs)
        sT[knext]  = x2T[knext] + y2T[knext]
        rT3[knext] = TaylorSeries.powHomogCoef(k, sT, 1.5, rT3, 0)
        Fx[knext]  = TaylorSeries.divHomogCoef(k, xT.coeffs, rT3, Fx, 0)
        Fy[knext]  = TaylorSeries.divHomogCoef(k, yT.coeffs, rT3, Fy, 0)
        # The equations of motion define the recurrencies
        xT.coeffs[knext+1]  = vxT.coeffs[knext] / knext
        yT.coeffs[knext+1]  = vyT.coeffs[knext] / knext
        vxT.coeffs[knext+1] = -GM * Fx[knext] / knext
        vyT.coeffs[knext+1] = -GM * Fy[knext] / knext
    end
    
    return Taylor1[ xT, yT, vxT, vyT ]
end